# CFM-ID Preprocessing Notebook

The purpose of this notebook is to convert the various CFM-ID outputs we generated during this project, and provide in this repository, into single MSPs that match the formatting generated by our evaluation scripts.

For fair comparison against our method and NEIMS, we also discard any peaks below the NIST-20 min m/z of 50Da.

We cloned CFM-ID from https://bitbucket.org/wishartlab/cfm-id-code/src and built it from source on the MIT Supercloud HPC cluster. For parallel processing we ran a SLURM batch job, using a single separate instance of CFM-ID per core per HPC node. We did this separately for positive and negative predictions, and invoked CFM-ID like so:

```
./bin/cfm-predict $INPUT_FILE 0.001 ./cfm-pretrained-models/'[M+H]+'/param_output.log ./cfm-pretrained-models/'[M+H]+'/param_config.txt 1 $OUTPUT_DIR 0
```

changing '[M+H]+' to '[M-H]-' as appropriate, and where INPUT_FILE is a text file of SMILES strings (formatted as appropriately for CFM-ID).

In [3]:
import numpy as np
import numpy.random as npr
import pandas as pd
from tqdm import tqdm

from rdkit import Chem, RDLogger
RDLogger.DisableLog('rdApp.*')

from pandarallel import pandarallel
from os import cpu_count
pandarallel.initialize(progress_bar=False, verbose=0, nb_workers=8)

import sys
sys.path.append('../..')

In [4]:
from src.io import read_msp, write_msp, read_cfm
from rdkit.Chem.rdMolDescriptors import CalcMolFormula
from rdkit.Chem.Descriptors import ExactMolWt

min_mz = 50

cols = ['Spectrum','SMILES','Precursor_type','NCE',
        'Instrument','InChIKey','Formula','PrecursorMZ']

def convert_cfmid(df):
    df = df.copy()
    
    df['NCE'] = df['Comment'].map({'Energy0': 20., 'Energy1': 35., 'Energy2': 50.})

    df['SMILES'] = df['Smiles/Inchi']
    df['mol'] = df['SMILES'].apply(Chem.MolFromSmiles)
    df['InChIKey'] = df['mol'].apply(Chem.MolToInchiKey)
    df['Formula'] = df['mol'].apply(CalcMolFormula)
    df['Precursor_type'] = df['Name'].str[0].map({'+':'[M+H]+','-':'[M-H]-'})
    df['PrecursorMZ'] = df['mol'].apply(ExactMolWt) + df['Precursor_type'].map({
        '[M+H]+': ExactMolWt(Chem.MolFromSmiles('[H]')),
        '[M-H]-': -ExactMolWt(Chem.MolFromSmiles('[H]')),
    })
    df['Instrument'] = 'Thermo Finnigan Elite Orbitrap' # placeholder

    for i, (xs, ys) in enumerate(zip(df.mzs,df.intensities)):
        mask = xs >= min_mz
        df.mzs.values[i] = xs[mask]
        df.intensities.values[i] = ys[mask]
        
    return df

# NIST-20

In [5]:
nist_df = pd.concat([
    read_msp('./nist-20/nist-20-positive.msp',parallel=True),
    read_msp('./nist-20/nist-20-negative.msp',parallel=True)
])

nist_df = convert_cfmid(nist_df)

nist_df = nist_df.loc[nist_df['mzs'].str.len()>0]

# we didn't use the identifiers when we generated these, so recover them
nist = pd.read_csv('../nist-20/hr_msms_nist_test.tsv',header=None,sep='\t')
nist.columns = ['Spectrum','SMILES','Precursor_type','NCE','Instrument']
nist['mols'] = nist['SMILES'].parallel_apply(lambda x: Chem.MolFromSmiles(x))
nist['InChIKey'] = nist['mols'].parallel_apply(lambda x: Chem.MolToInchiKey(x))

nist_df = nist_df.merge(nist,on=['InChIKey','Precursor_type','NCE'],suffixes=('_',''))

write_msp(
    'nist-20/hr_msms_nist_test_cfm-id.msp',
    nist_df['mzs'].tolist(), 
    nist_df['intensities'].tolist(),
    **{c: nist_df[c].tolist() for c in cols}
)

# CASMI-16

In [ ]:
casmi_msps = !cat <(ls -1 ./casmi-16/Training-*.msp) <(ls -1 ./casmi-16/Challenge-*.msp)

casmi_df = pd.concat([*pd.Series(casmi_msps).parallel_apply(read_msp)])

casmi_df = convert_cfmid(casmi_df)

# CASMI uses CE stepping; approximate by only using the middle NCE
casmi_df = casmi_df.query('NCE==35')

casmi_df['Spectrum'] = casmi_df['filename'].str.split('.').str[0]

write_msp(
    'casmi-16/casmi-16_cfm-id.msp',
    casmi_df['mzs'].tolist(), 
    casmi_df['intensities'].tolist(),
    **{c: casmi_df[c].tolist() for c in cols}
)

# GNPS

In [ ]:
pos_fns = !find gnps/gnps-pos -name "*.log"
neg_fns = !find gnps/gnps-neg -name "*.log"

gnps_pos_data = []
for fn in tqdm(pos_fns):
    gnps_pos_data.extend(read_cfm(fn))
gnps_pos_df = pd.DataFrame(gnps_pos_data)
gnps_pos_df['Precursor_type'] = '[M+H]+'

gnps_neg_data = []
for fn in tqdm(neg_fns):
    gnps_neg_data.extend(read_cfm(fn))
gnps_neg_df = pd.DataFrame(gnps_neg_data)
gnps_neg_df['Precursor_type'] = '[M-H]-'

gnps_df = pd.concat([gnps_pos_df,gnps_neg_df])

gnps_df['NCE'] = gnps_df['energy'].map({'energy0': 20., 'energy1': 35., 'energy2': 50.})

gnps_df = gnps_df.query('NCE==35')

gnps_df['SMILES'] = gnps_df['smiles']
gnps_df['mol'] = gnps_df['SMILES'].parallel_apply(lambda x: Chem.MolFromSmiles(x))
gnps_df['InChIKey'] = gnps_df['mol'].parallel_apply(lambda x: Chem.MolToInchiKey(x))
gnps_df['Formula'] = gnps_df['mol'].apply(CalcMolFormula)
gnps_df['PrecursorMZ'] = gnps_df['mol'].apply(ExactMolWt) + gnps_df['Precursor_type'].map({
    '[M+H]+': ExactMolWt(Chem.MolFromSmiles('[H]')),
    '[M-H]-': -ExactMolWt(Chem.MolFromSmiles('[H]')),
})
gnps_df['Instrument'] = 'Thermo Finnigan Elite Orbitrap' # placeholder

for i, (xs, ys) in tqdm(enumerate(zip(gnps_df.mzs,gnps_df.intensities))):
    mask = xs >= min_mz
    gnps_df.mzs.values[i] = xs[mask]
    gnps_df.intensities.values[i] = ys[mask]

gnps = pd.read_csv('../gnps/gnps.tsv',header=None,sep='\t')
gnps.columns = ['Spectrum','SMILES','Precursor_type','NCE','Instrument']
gnps['mols'] = gnps['SMILES'].apply(Chem.MolFromSmiles)
gnps['InChIKey'] = gnps['mols'].apply(Chem.MolToInchiKey)

gnps_df = gnps_df.merge(gnps,on=['InChIKey','Precursor_type','NCE'],suffixes=('_',''))
gnps_df = gnps_df.drop_duplicates(subset='Spectrum', keep='first')

write_msp(
    'gnps/gnps_cfm-id.msp',
    gnps_df['mzs'].tolist(), 
    gnps_df['intensities'].tolist(),
    **{c: gnps_df[c].tolist() for c in cols}
)

# ChEMBL

In [6]:
# this is system dependent -- many loose files here. decompress to your location of choice
# and set (pos,neg)_fns appropriately

!cp -n chembl/chembl_decoys*.tar.gz $TMPDIR/
!cd $TMPDIR; for f in chembl_decoys-*.tar.gz; do tar xzf $f; done

pos_fns = !find $TMPDIR/chembl_decoys-pos -name "*.log"
neg_fns = !find $TMPDIR/chembl_decoys-neg -name "*.log"

len(pos_fns), len(neg_fns)

(199998, 199992)

In [7]:
from src.io import read_cfm

chembl_data = pd.Series(pos_fns+neg_fns).parallel_apply(read_cfm)
xs = []
for x in chembl_data:
    xs.extend(x)
chembl_df = pd.DataFrame(xs)

chembl_df['Precursor_type'] = None
chembl_df['Precursor_type'].iloc[:len(pos_fns)] = '[M+H]+'
chembl_df['Precursor_type'].iloc[len(pos_fns):] = '[M-H]-'

chembl_df['NCE'] = chembl_df['energy'].map({'energy0': 20., 'energy1': 35., 'energy2': 50.})

chembl_df['SMILES'] = chembl_df['smiles']
chembl_df['mol'] = chembl_df['SMILES'].parallel_apply(lambda x: Chem.MolFromSmiles(x))
chembl_df['InChIKey'] = chembl_df['mol'].parallel_apply(lambda x: Chem.MolToInchiKey(x))
chembl_df['Formula'] = chembl_df['mol'].apply(CalcMolFormula)
chembl_df['PrecursorMZ'] = chembl_df['mol'].apply(ExactMolWt)
chembl_df['PrecursorMZ'] += chembl_df['Precursor_type'].map({
    '[M+H]+': ExactMolWt(Chem.MolFromSmiles('[H]')),
    '[M-H]-': -ExactMolWt(Chem.MolFromSmiles('[H]')),
})
chembl_df['Instrument'] = 'Thermo Finnigan Elite Orbitrap' # placeholder

for i, (xs, ys) in tqdm(enumerate(zip(chembl_df.mzs,chembl_df.intensities))):
    mask = xs >= min_mz
    chembl_df.mzs.values[i] = xs[mask]
    chembl_df.intensities.values[i] = ys[mask]

chembl_df = chembl_df.reset_index(drop=True)
chembl_df['Spectrum'] = 'decoy_' + chembl_df.index.astype(str)

# concatenate the nist predictions to get full library
chembl_df = pd.concat([chembl_df,nist_df])

chembl_df = chembl_df.loc[chembl_df['mzs'].str.len()>0]

write_msp(
    'chembl/nist-20_chembl_decoys_cfm-id.msp',
    chembl_df['mzs'].tolist(), 
    chembl_df['intensities'].tolist(),
    **{c: chembl_df[c].tolist() for c in cols}
)

1199970it [00:16, 74337.93it/s]
